# **다음 분기에 어떤 게임을 설계해야 할까?**

> `Name` : 게임의 이름입니다.  
`Platform` : 게임이 지원되는 플랫폼의 이름입니다.  
`Year` : 게임이 출시된 연도입니다.  
`Genre` : 게임의 장르입니다.  
`Publisher` : 게임을 제작한 회사입니다.  
`NA_Sales` : 북미지역에서의 출고량입니다.  
`EU_Sales` : 유럽지역에서의 출고량입니다.  
`JP_Sales` : 일본지역에서의 출고량입니다.  
`Other_Sales` : 기타지역에서의 출고량입니다.  


> - 지역에 따라서 선호하는 게임 장르가 다를까   
- 연도별 게임의 트렌드가 있을까  
- 출고량이 높은 게임에 대한 분석 및 시각화 프로세스





# **1. 전처리**




In [ ]:
# 데이터 불러오기

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('vgames2.csv', index_col=0)
data.sample(10)

In [ ]:
# 데이터 복사
df = data.copy()

In [ ]:
# 데이터 결측치, 타입, 행/렬 개수 확인
df.info()

# 전처리 해야 할 것.

> 

- NA_Sales, EU_Sales, JP_Sales, Other_Sales 컬럼에 들어가 있는 숫자가 아닌 문자열을 제거하고 float 형태로 바꿔야 한다. 
- Genre 컬럼의 결측치를 처리해야 한다.  
- Publisher 컬럼의 결측치를 처리해야 한다.
- Year 컬럼의 결측치를 처리하고, 타입을 int로 바꿔야 한다.  
- 각 컬럼의 이상치를 제거해야 한다.

In [ ]:
# NA_Sales, EU_Sales, JP_Sales, Other_Sales 컬럼에 들어가 있는 숫자가 아닌 문자열을 제거하고 float 형태로 바꿔야 한다.
for column in df.iloc[:,5:].columns:
  df[column][df[column].str.contains('M')] = df[column][df[column].str.contains('M')].str.replace('M', '')
  df[column][df[column].str.contains('K')] = df[column][df[column].str.contains('K')].str.replace('K', '').astype(float)/1000
  df[column] = df[column].astype(float)

In [ ]:
# Genre 컬럼의 결측치를 처리해야 한다.
# 데이터의 샘플링 해본 결과, 이름이 같으면 장르도 같은 것을 확인했으므로 이름이 같은 게임 행이 존재 하는지 확인하고 있으면 장르 결측치를 채워준다.

# 이름이 겹치는 행들을 데이터프레임으로 추출.
df[df.duplicated(subset=['Name'], keep=False)]
# df_duplicate에 할당.
df_duplicate = df[df.duplicated(subset=['Name'], keep=False)]
# 여기서 Genre 컬럼의 결측치 확인
df_duplicate[df_duplicate.Genre.isnull()]
# 결측치가 있는 행을 인덱스로 추출하여 genre_nan에 할당
genre_nan = df_duplicate[df_duplicate.Genre.isnull()].index

In [ ]:
# 채워줄 장르 결측치 값을 리스트에 저장
genre_change = []

for index in genre_nan:
  change = df_duplicate.query('Name == @df_duplicate.Name[@index]')[df_duplicate.query('Name == @df_duplicate.Name[@index]').Genre.notnull()].iloc[0,3]
  genre_change.append(change)

In [ ]:
# 결측치 대체
df.loc[genre_nan,['Genre']] = genre_change

In [ ]:
# Genre 컬럼의 나머지 결측치와 Publisher 컬럼의 결측치는 대체할 수 있는 값이 없으므로 제거함.
df = df.drop(df[df.Genre.isnull() | df.Publisher.isnull()].index).reset_index(drop=True)

In [ ]:
df.describe()

In [ ]:
# Year의 결측치 중 출고량이 큰 값들이므로 제거하지 않고, 대체해준다.
df[df.Year.isnull()].query('NA_Sales > 0.5 | EU_Sales > 0.5 | JP_Sales > 0.5 | Other_Sales > 0.5')
nan_Year = df[df.Year.isnull()].query('NA_Sales > 0.5 | EU_Sales > 0.5 | JP_Sales > 0.5 | Other_Sales > 0.5').index

In [ ]:
# 구글 서칭으로 직접 데이터 가져옴.
Year_change = [2007, 2011, 2005, 2001, 1997, 2008, 1999, 2002, 2008, 2010, 2002, 1980, 2003, 2008, 2011, 2008, 2006, 2007, 2003, 1977, 2011, 2007, 2010, 2001, 2002, 2006, 2008, 1977, 2011,1980, 2010]

In [ ]:
# 결측치 값 대체함.
df.loc[nan_Year,['Year']] = Year_change

In [ ]:
# 나머지 Year 컬럼의 결측치 값은 제거해줌
df = df.drop(df[df.Year.isnull()].index).reset_index(drop=True)

In [ ]:
df

In [ ]:
# Year 컬럼의 타입을 int로 바꿔주고, 이상치 제거.
df.Year = df.Year.astype(int)
plt.hist(df.Year)
plt.xlim([1750,2020])

In [ ]:
# 1800년 이하의 값들은 제거
df = df.drop(df.query('Year < 1800').index).reset_index(drop=True)

In [ ]:
df.describe()

# **2. 지역에 따라서 선호하는 게임 장르가 다를까**

In [ ]:
# 지역 별 장르에 따른 출고량 합
df.groupby('Genre').sum().iloc[:,1:]

In [ ]:
df_region = df.groupby('Genre').sum().iloc[:,1:];

In [ ]:
plt.figure(figsize=(30,15))
plt.subplot(2,2,1)
sns.barplot(df_region.index, df_region.NA_Sales, label='NA')
plt.title('NA', size=20)
plt.subplot(2,2,2)
sns.barplot(df_region.index, df_region.EU_Sales, label='EU')
plt.title('EU', size=20)
plt.subplot(2,2,3)
sns.barplot(df_region.index, df_region.JP_Sales, label='JP')
plt.title('JP', size=20)
plt.subplot(2,2,4)
sns.barplot(df_region.index, df_region.Other_Sales, label='Other')
plt.title('Other', size=20)
plt.show()

1. 북미는 액션 장르의 게임을 가장 선호하고 뒤이어, 스포츠와 슈팅게임을 선호한다.
2. 유럽은 액션 장르의 게임을 가장 선호하고 뒤이어, 스포츠와 슈팅게임을 선호한다.
3. 일본은 RPG 장르의 게임을 가장 선호하고 뒤이어, 액션게임을 선호한다.
4. 그 외의 지역에서는 액션 장르의 게임을 가장 선호하고 뒤이어, 스포츠와 슈팅게임을 선호한다.

# 3.연도별 게임의 트렌드가 있을까

In [ ]:
df_trend = df.sort_values(by = ['Year', 'Genre', 'Platform']).iloc[:,1:4].reset_index(drop=True)
df_trend

In [ ]:
uni = df_trend.Year.unique()

In [ ]:
plt.figure(figsize=(50,30))
plt.subplot(2,1,1)
sns.countplot(df_trend.query('Year <= 1998').Year, hue = df_trend.query('Year <= 1998').Genre, dodge=False, palette='Dark2')
plt.legend(loc='upper left', fontsize=25)
plt.subplot(2,1,2)
sns.countplot(df_trend.query('Year > 1998').Year, hue = df_trend.query('Year > 1998').Genre, dodge=False, palette='Dark2')
plt.legend(loc='upper left', fontsize=25)

In [ ]:
plt.figure(figsize=(50,30))
plt.subplot(2,1,1)
sns.countplot(df_trend.query('Year <= 1998').Year, hue = df_trend.query('Year <= 1998').Platform, dodge=False, palette='Dark2')
plt.legend(loc='upper left', fontsize=20)
plt.subplot(2,1,2)
sns.countplot(df_trend.query('Year > 1998').Year, hue = df_trend.query('Year > 1998').Platform, dodge=False, palette='Dark2')
plt.legend(loc='upper right', fontsize=20)

In [ ]:
df.query('Year == 2009').Genre.value_counts()

# 3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스

In [ ]:
df['Sum_Sales'] = df.NA_Sales + df.EU_Sales + df.JP_Sales + df.Other_Sales

In [ ]:
best_df = df.sort_values(by = 'Sum_Sales', ascending=False).iloc[0:50, :].reset_index(drop=True)
best_df

In [ ]:
plt.figure(figsize=(20,15))
plt.pie(best_df.Genre.value_counts(), labels=best_df.Genre.value_counts().index, autopct='%.1f%%')

In [ ]:
plt.figure(figsize=(20,15))
plt.pie(best_df.Platform.value_counts(), labels=best_df.Platform.value_counts().index, autopct='%.1f%%')